In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from zipfile import ZipFile
file_name = "drive/My Drive/ashrae-energy-prediction.zip"
with ZipFile(file_name, 'r') as zip:  
    zip.printdir() 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!') 

File Name                                             Modified             Size
building_metadata.csv                          2019-10-10 17:20:04        45527
sample_submission.csv                          2019-10-10 17:20:12    447562511
test.csv                                       2019-10-10 17:20:46   1462461085
train.csv                                      2019-10-10 17:22:42    678616640
weather_test.csv                               2019-10-10 17:23:40     14787908
weather_train.csv                              2019-10-10 17:23:42      7450075
Extracting all the files now...
Done!


In [3]:
!pip install category_encoders

     |████████████████████████████████| 102kB 3.9MB/s 


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
from IPython import display
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
import category_encoders as ce

In [0]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
      print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [6]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
weather_train = pd.read_csv("weather_train.csv")
weather_test = pd.read_csv("weather_test.csv")
building = pd.read_csv("building_metadata.csv")
sub = pd.read_csv("sample_submission.csv")
gc.collect()

0

In [0]:
le = LabelEncoder()
building["primary_use"] = le.fit_transform(building["primary_use"])

In [0]:
# Interpolation for missing values
weather_train = weather_train.groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both'))
gc.collect()
weather_test = weather_test.groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both'))

In [0]:
train = train.merge(building, on = "building_id", how = "left")
test = test.merge(building, on = "building_id", how = "left")

In [0]:
train = train.merge(weather_train, on = ["site_id", "timestamp"], how='left')
test = test.merge(weather_test, on = ["site_id", "timestamp"], how='left')

In [0]:
# Handling missing values
# Training set
train["year_built"].fillna(-1,inplace=True)
train["floor_count"].fillna(-1,inplace=True)
train["air_temperature"].fillna(train["air_temperature"].mean(),inplace=True)
train["dew_temperature"].fillna(train["dew_temperature"].mean(),inplace=True)
train["wind_speed"].fillna(train["wind_speed"].mean(),inplace=True)
train["wind_direction"].fillna(train["wind_direction"].mode(),inplace=True)
train.fillna(-999,inplace=True)
# Test set
test["year_built"].fillna(-1,inplace=True)
test["floor_count"].fillna(-1,inplace=True)
test["air_temperature"].fillna(test["air_temperature"].mean(),inplace=True)
test["dew_temperature"].fillna(test["dew_temperature"].mean(),inplace=True)
test["wind_direction"].fillna(test["wind_direction"].mean(),inplace=True)
test["wind_direction"].fillna(test["wind_direction"].mode(),inplace=True)
test.fillna(-999,inplace=True)

In [12]:
# Feature engineering
train["day"] = pd.to_datetime(train["timestamp"]).dt.day
train["month"] = pd.to_datetime(train["timestamp"]).dt.month
train["dayofweek"] = pd.to_datetime(train["timestamp"]).dt.dayofweek
train["hour"] = pd.to_datetime(train["timestamp"]).dt.hour
test["day"] = pd.to_datetime(test["timestamp"]).dt.day
test["month"] = pd.to_datetime(test["timestamp"]).dt.month
test["dayofweek"] = pd.to_datetime(test["timestamp"]).dt.dayofweek
test["hour"] = pd.to_datetime(test["timestamp"]).dt.hour

a_train = train["site_id"].astype(str) + "_" + train["building_id"].astype(str)
a_test = test["site_id"].astype(str) + "_" + test["building_id"].astype(str)
pbar = tqdm(total=8)
woe1 = ce.woe.WOEEncoder()
train["site_id-building_id_yearbuilt_woe"] = woe1.fit_transform(a_train,train["year_built"].apply(lambda x: 0 if x==-1 else 1))
pbar.update(1)
test["site_id-building_id_yearbuilt_woe"] = woe1.transform(a_test,test["year_built"].apply(lambda x: 0 if x==-1 else 1))
pbar.update(1)
woe2 = ce.woe.WOEEncoder()
train["site_id-building_id_floorcount_woe"] = woe2.fit_transform(a_train,train["floor_count"].apply(lambda x: 0 if x==-1 else 1)) 
pbar.update(1)
test["site_id-building_id_floorcount_woe"] = woe2.transform(a_test,test["floor_count"].apply(lambda x: 0 if x==-1 else 1))
pbar.update(1)
woe3 = ce.woe.WOEEncoder()
train["primaryuse_yearbuilt_woe"] = woe3.fit_transform(train["primary_use"].astype(str),train["year_built"].apply(lambda x: 0 if x==-1 else 1)) 
pbar.update(1)
test["primaryuse_yearbuilt_woe"] = woe3.transform(test["primary_use"].astype(str),test["year_built"].apply(lambda x: 0 if x==-1 else 1))
pbar.update(1)
woe4 = ce.woe.WOEEncoder()
train["primaryuse_floorcount_woe"] = woe4.fit_transform(train["primary_use"].astype(str),train["floor_count"].apply(lambda x: 0 if x==-1 else 1)) 
pbar.update(1)
test["primaryuse_floorcount_woe"] = woe4.transform(test["primary_use"].astype(str),test["floor_count"].apply(lambda x: 0 if x==-1 else 1))
pbar.update(1)
pbar.close()

100%|██████████| 8/8 [03:29<00:00, 31.08s/it]


In [0]:
del weather_train
del weather_test
del building

In [14]:
gc.collect()

0

In [15]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)
sub = reduce_mem_usage(sub)

Mem. usage decreased to 1777.49 Mb (60.5% reduction)
Mem. usage decreased to 2346.19 Mb (70.5% reduction)
Mem. usage decreased to 198.83 Mb (68.7% reduction)


In [16]:
gc.collect()

0

In [0]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
dates_range = pd.date_range(start='2015-12-31', end='2019-01-01')
us_holidays = calendar().holidays(start=dates_range.min(), end=dates_range.max())
train['is_holiday'] = (pd.to_datetime(train['timestamp']).dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)
test['is_holiday'] = (pd.to_datetime(test['timestamp']).dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)

train.loc[(train['dayofweek'] == 5) | (train['dayofweek'] == 6) , 'is_holiday'] = 1
test.loc[(test['dayofweek']) == 5 | (test['dayofweek'] == 6) , 'is_holiday'] = 1

In [0]:
train = train.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

In [0]:
def degToCompass(num):
    val=int((num/22.5)+.5)
    arr=[i for i in range(0,16)]
    return arr[(val % 16)]

In [0]:
train['wind_direction'] = train['wind_direction'].apply(degToCompass)
test['wind_direction'] = test['wind_direction'].apply(degToCompass)

In [0]:
target = np.log1p(train["meter_reading"])
train.drop(["timestamp", "meter_reading"], axis = 1, inplace=True)
test.drop(["timestamp", "row_id"], axis = 1, inplace=True)

In [22]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)
sub = reduce_mem_usage(sub)

Mem. usage decreased to 890.63 Mb (13.0% reduction)
Mem. usage decreased to 1869.00 Mb (13.0% reduction)
Mem. usage decreased to 198.83 Mb (0.0% reduction)


In [23]:
train.shape, target.shape, test.shape

((19869988, 23), (19869988,), (41697600, 23))

In [24]:
!pip install optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import random
from tqdm import tqdm
import optuna
from statistics import mean 

     |████████████████████████████████| 122kB 5.2MB/s 
     |████████████████████████████████| 1.1MB 65.7MB/s 
     |████████████████████████████████| 81kB 13.3MB/s 
     |████████████████████████████████| 471kB 70.3MB/s 
     |████████████████████████████████| 112kB 63.0MB/s 
     |████████████████████████████████| 51kB 9.4MB/s 
     |████████████████████████████████| 61kB 10.4MB/s 
  Created wheel for optuna: filename=optuna-0.18.1-cp36-none-any.whl size=165105 sha256=6c3ba0d28662e67c426437e1684bf16551239cbcd5575f71ee63542c246e23da
  Stored in directory: /root/.cache/pip/wheels/40/d2/bd/49b16e74a177e74ea789e677aef515981f021e1979061c4a51
  Created wheel for alembic: filename=alembic-1.3.0-py2.py3-none-any.whl size=144427 sha256=609421d1ab6ba0b92d915932f34ee5449bbaba77ed69824d6a83578ad3928fb1
  Stored in directory: /root/.cache/pip/wheels/40/f8/22/ad0f408796a4c656fae5ee1fd8d8a139b19ca4af61059cea5b
  Created wheel for Mako: filename=Mako-1.1.0-cp36-none-any.whl size=75363 sha256=a87103a

In [26]:
def objective(trial):
    gc.collect()
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, 'l2')
    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'l2',
        'num_leaves': trial.suggest_int('num_leaves', 2, 250),
        'learning_rate': 0.5,
        'subsample': 0.25,
        'subample_freq': 1,
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        'bagging_freq': 5,
        'verbose': 1,
    }
    X_train, X_valid, y_train, y_valid = train_test_split(train,target,test_size=0.1, random_state=random.randint(1,1000), shuffle=True)
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_valid, label=y_valid)
    model = lgb.train(params,
                  dtrain,
                  num_boost_round=500,
                  valid_sets=dvalid,
                  early_stopping_rounds=5,
                  verbose_eval=100,
                  callbacks=[pruning_callback])
    gc.collect()
    i=0
    res=[]
    step_size = 1000000
    for j in tqdm(range(int(np.ceil(X_valid.shape[0]/1000000)))):
        res.append(model.predict(X_valid.iloc[i:i+step_size],num_iteration=model.best_iteration))
        i+=step_size
    preds = np.concatenate(res)
    return np.sqrt(mean_squared_error(y_valid,preds))

study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=10))
study.optimize(objective, n_trials=60)

print(study.best_params)

Training until validation scores don't improve for 5 rounds.
[100]	valid_0's l2: 0.765886
[200]	valid_0's l2: 0.644599
[300]	valid_0's l2: 0.571486
[400]	valid_0's l2: 0.533681
[500]	valid_0's l2: 0.513203
Did not meet early stopping. Best iteration is:
[500]	valid_0's l2: 0.513203


100%|██████████| 2/2 [00:17<00:00,  8.99s/it]
[I 2019-11-08 13:55:28,750] Finished trial#0 resulted in value: 0.7163815824000324. Current best value is 0.7163815824000324 with parameters: {'num_leaves': 94, 'lambda_l1': 4.282283074936329e-08, 'lambda_l2': 3.012123700190518e-07, 'feature_fraction': 0.5640724977988825, 'bagging_fraction': 0.1446946218093187}.


Training until validation scores don't improve for 5 rounds.
[100]	valid_0's l2: 0.768765
[200]	valid_0's l2: 0.622889
[300]	valid_0's l2: 0.555603
[400]	valid_0's l2: 0.517559
[500]	valid_0's l2: 0.49522
Did not meet early stopping. Best iteration is:
[500]	valid_0's l2: 0.49522


100%|██████████| 2/2 [00:17<00:00,  8.97s/it]
[I 2019-11-08 13:59:44,480] Finished trial#1 resulted in value: 0.703718717329. Current best value is 0.703718717329 with parameters: {'num_leaves': 94, 'lambda_l1': 1.1220986590670759e-07, 'lambda_l2': 1.841575268234306e-07, 'feature_fraction': 0.5412225682986602, 'bagging_fraction': 0.28808310320882985}.


Training until validation scores don't improve for 5 rounds.
[100]	valid_0's l2: 0.74681
[200]	valid_0's l2: 0.602207
[300]	valid_0's l2: 0.548454
[400]	valid_0's l2: 0.516646
[500]	valid_0's l2: 0.495399
Did not meet early stopping. Best iteration is:
[500]	valid_0's l2: 0.495399


...

Training until validation scores don't improve for 5 rounds.
[100]	valid_0's l2: 0.468668
[200]	valid_0's l2: 0.381852
[300]	valid_0's l2: 0.345248
[400]	valid_0's l2: 0.32127
[500]	valid_0's l2: 0.306155
Did not meet early stopping. Best iteration is:
[500]	valid_0's l2: 0.306155


100%|██████████| 2/2 [00:25<00:00, 12.84s/it]
[I 2019-11-08 18:01:27,346] Finished trial#58 resulted in value: 0.5533131079128342. Current best value is 0.5508572112487604 with parameters: {'num_leaves': 250, 'lambda_l1': 1.1950069081340828e-06, 'lambda_l2': 0.4863008634602687, 'feature_fraction': 0.9543235876804018, 'bagging_fraction': 0.9821595681875143}.


Training until validation scores don't improve for 5 rounds.


[I 2019-11-08 18:02:21,222] Setting status of trial#59 as TrialState.PRUNED. Trial was pruned at iteration 11.


{'num_leaves': 250, 'lambda_l1': 1.1950069081340828e-06, 'lambda_l2': 0.4863008634602687, 'feature_fraction': 0.9543235876804018, 'bagging_fraction': 0.9821595681875143}


In [27]:
print(study.best_params)

{'num_leaves': 250, 'lambda_l1': 1.1950069081340828e-06, 'lambda_l2': 0.4863008634602687, 'feature_fraction': 0.9543235876804018, 'bagging_fraction': 0.9821595681875143}


In [0]:
best = study.best_params
best['learning_rate'] = 0.5
best['objective'] = 'regression'
best['subsample'] = 0.25
best['metric'] = 'l2'
best['verbose'] = 1

In [29]:
model = lgb.train(best,lgb.Dataset(train, label=target),verbose_eval=True,num_boost_round=500)
i=0
res=[]
step_size = 1000000
for j in tqdm(range(int(np.ceil(test.shape[0]/1000000)))):
    res.append(np.expm1(model.predict(test.iloc[i:i+step_size],num_iteration=model.best_iteration)))
    i+=step_size
print("Saving!")
sub["meter_reading"] = np.concatenate(res)
sub["meter_reading"][sub["meter_reading"] < 0] = 0
sub.to_csv("submission.csv", index = False)

100%|██████████| 42/42 [07:51<00:00, 10.65s/it]


Saving!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [30]:
sub.head()

,row_id,meter_reading
0,0,138.666427
1,1,174.007271
2,2,15.618977
3,3,388.048606
4,4,2449.129995


In [31]:
best

{'bagging_fraction': 0.9821595681875143,
 'feature_fraction': 0.9543235876804018,
 'lambda_l1': 1.1950069081340828e-06,
 'lambda_l2': 0.4863008634602687,
 'learning_rate': 0.5,
 'metric': 'l2',
 'num_leaves': 250,
 'objective': 'regression',
 'subsample': 0.25,
 'verbose': 1}